In [23]:
import pandas as pd
import requests
import yfinance as yf
from bs4 import BeautifulSoup
from pymongo import MongoClient

In [24]:
class Accion:
    def __init__(self, arrayaccion) -> None:
        pass

class Origen:
    def actualizarBD(self):
        bdMongo = MongoClient('localhost')
        print(bdMongo.list_database_names())
        pass
    def __visualizarBD(self):
        pass
    def __cotizacionMinima(self):
        pass
    def __cotizacionMaxima(self):
        pass
    def __filtrarAcciones(self):
        pass
    def accion(self, symbol):
        return Accion(self.__filtrarAcciones(symbol))

origen = Origen()
origen.actualizarBD()


['admin', 'config', 'local']


In [25]:
       
class bolsaMadrid(Origen):

   def __init__(self):
       pass
   
   def obtenerDatos(self):
    #Hacemos la consulta a la pagina y guardamos el contenido en una variable soup
    url_page = 'https://www.bolsamadrid.es/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000'
    page = requests.get(url_page).text 
    soup = BeautifulSoup(page, "lxml")

    #Buscamos la tabla de donde obtener los datos
    tabla = soup.find('table', attrs={'id': 'ctl00_Contenido_tblAcciones'})

    #creamos una lista vacia que llenaremos con los datos de las tablas
    columnas = []

    #buscamos los encabezados de la tabla y los agregamos a la lista columnos (deberia ubicarse en la posicion 0)
    header = []
    for th in tabla.find_all('th'):
        header.append(th.text)
    columnas.append(header)

    for tr in tabla.find_all('tr'):
        fila = []
        for td in tr.find_all('td'):
            fila.append(td.text)
        if len(fila) > 0:
            columnas.append(fila)
            
    #creamos el dataframe usando la lista columnas,pasamos el contenido (de la posicion 1 en adelante), y lo encabezados en (posicion 0)
    df = pd.DataFrame(columnas[1:], columns=columnas[0])

    #Reemplazamos la , por un . y convertimos en float
    df["% Dif."] = df["% Dif."].str.replace(",", ".").astype(float)
    df["Máx."] = df["Máx."].str.replace(",", ".").astype(float)
    df["Mín."] = df["Mín."].str.replace(",", ".").astype(float)
    df["Últ."] = df["Últ."].str.replace(",", ".").astype(float)
    
    #Guardamos en el archivo csv

    df.to_csv('bolsaMadrid.csv')

   def mostrarDatosEnPantalla(self, acciones):
   #Mostrar tabla en pantalla usando BeautiFoul Soup
      print('')
      print('TABLA DE DATOS BOLSA DE MADRID (USANDO BS4)')
      df_bolsaMadrid = pd.read_csv('bolsaMadrid.csv')
      df_bolsaMadrid = df_bolsaMadrid[df_bolsaMadrid.Nombre.isin(acciones)]
      print(df_bolsaMadrid)

class yfinance():

    def __init__(self):
       pass
    def obtenerDatos(acciones):
        df_list = list()
        for ticker in acciones:
            data = yf.download(ticker, group_by='ticker', period='1d')
            data['ticker'] = ticker  
            df_list.append(data)

        df = pd.concat(df_list)

        # save to csv
        df.to_csv('yahoo_finance.csv')

class investpy():

   def __init__(self):
      pass

   def extraerDatos():
      pass


class Cotizaciones(bolsaMadrid, yfinance, investpy):

    def __init__(self, acciones):
        self.acciones = acciones
        self.actualizarBD()
       

    def actualizarBD(self):
        bolsaMadrid.obtenerDatos(self)
        yfinance.obtenerDatos(self.acciones)
        ##investpy.obtenerDatos(self)

    def leerDatos():
        pass

In [26]:
acciones = ['ACCIONA', 'BBVA', 'GRIFOLS CL.A', 'PEPITO', 'REPSOL', 'ENDESA']
cotizaciones = Cotizaciones(acciones)
cotizaciones.mostrarDatosEnPantalla()


[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ACCIONA: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  2 of 2 completed

2 Failed downloads:
- GRIFOLS: No data found, symbol may be delisted
- CL.A: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- PEPITO: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- REPSOL: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ENDESA: No data found, symbol may be delisted


TypeError: mostrarDatosEnPantalla() missing 1 required positional argument: 'acciones'